In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import os
from google.colab import files


In [ ]:
# Set Constants
IMG_SIZE = (224, 224)  # Standard size for CNN models
BATCH_SIZE = 16
EPOCHS = 10
NUM_GAZE_IMAGES = 9  # Each sample consists of 9 images


In [ ]:

def split_9gaze_image(image, grid_size=(3, 3)):
    """Splits a single 9-gaze image into 9 individual images."""
    h, w, _ = image.shape
    grid_h, grid_w = grid_size
    img_h, img_w = h // grid_h, w // grid_w  # Calculate individual image dimensions

    images = []
    for i in range(grid_h):
        for j in range(grid_w):
            cropped_img = image[i * img_h:(i + 1) * img_h, j * img_w:(j + 1) * img_w]
            images.append(cropped_img)

    return images  # Returns a list of 9 images



# Load Data
def load_images_from_folder(folder):
    images = []
    labels = []  # Modify based on dataset structure
    for subject_folder in os.listdir(folder):  # Assuming each subject has a folder containing 9 images
        subject_path = os.path.join(folder, subject_folder)
        subject_images = []
        if os.path.isdir(subject_path):
            for filename in sorted(os.listdir(subject_path)):  # Ensure consistent order of images
                img_path = os.path.join(subject_path, filename)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, IMG_SIZE)
                    img = img / 255.0  # Normalize
                    subject_images.append(img)
            if len(subject_images) == NUM_GAZE_IMAGES:
                images.append(subject_images)
                labels.append(0)  # Replace with actual labels
    return np.array(images), np.array(labels)



# define CNN Model for Multi-Image Input
def baseline_model():
    input_layer = layers.Input(shape=(NUM_GAZE_IMAGES, IMG_SIZE[0], IMG_SIZE[1], 3))

    # Apply CNN to each image separately using TimeDistributed
    base_cnn = keras.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Flatten()
    ])

    cnn_features = layers.TimeDistributed(base_cnn)(input_layer)

    # LSTM layer to capture sequential dependencies across 9-gaze images
    lstm_layer = layers.LSTM(64, return_sequences=False)(cnn_features)

    # Dense layers for final classification
    dense_layer = layers.Dense(128, activation='relu')(lstm_layer)
    dense_layer = layers.Dropout(0.5)(dense_layer)
    output_layer = layers.Dense(1, activation='sigmoid')  # Binary classification

    model = keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/My Drive/AbnEye")

In [ ]:

# Load and Preprocess Data (Modify path)
train_images, train_labels = load_images_from_folder("/content/drive/My Drive/AbnEye/train")
test_images, test_labels = load_images_from_folder(folder="/content/drive/My Drive/AbnEye/test")

# training Model
model = baseline_model()
model.fit(train_images, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(test_images, test_labels))

# Save Model
model.save("strabismus_baseline_model.h5")

# Evaluate Model
loss, acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {acc * 100:.2f}%")

NotADirectoryError: [Errno 20] Not a directory: 'train/image0.jpeg'